In [96]:
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

#read data and label from text file
#remove any url in data if any.
import re
#text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)

X = []
with open("dataset/data.txt", "r") as f:
  for line in f:
    line = re.sub(r'^https?:\/\/.*[\r\n]*', '', line, flags=re.MULTILINE)
    X.append(line.strip().replace('\\',''))

Y = []
with open("dataset/label.txt", "r") as f:
  for line in f:
    Y.append(line.strip())
    
Labels = []
for i in Y:
    if i == 'Animal Science':
        Labels.append(1)
    elif i == 'Anthropology':
        Labels.append(2)
    elif i == 'Astronomy':
        Labels.append(3)
    elif i == 'Biology':
        Labels.append(4)
    elif i == 'Cancer':
        Labels.append(5)
    elif i == 'Chemistry':
        Labels.append(6)
    elif i == 'Computer Science':
        Labels.append(7)
    #elif i == 'Deaths + injuries':
    #    Labels.append(8)
    elif i == 'Earth Science':
        Labels.append(8)
    elif i == 'Engineering':
        Labels.append(9)
    elif i == 'Environment':
        Labels.append(10)
    elif i == 'Epidemiology':
        Labels.append(11)
    elif i == 'Geology':
        Labels.append(12)
    elif i == 'Health':
        Labels.append(13)
    elif i == 'Mathematics':
        Labels.append(14)
    elif i == 'Medicine':
        Labels.append(15)
    #elif i == 'Misleading Title':
    #    Labels.append(17)
    elif i == 'Nanoscience':
        Labels.append(16)
    elif i == 'Neuroscience':
        Labels.append(17)
    elif i == 'None':
        Labels.append(18)
    elif i == 'Paleontology':
        Labels.append(19)
    #elif i == 'PDF - Psychology':
    #    Labels.append(22)
    elif i == 'Physics':
        Labels.append(20)
    elif i == 'Psychology':
        Labels.append(21)
    elif i == 'Social Science':
        Labels.append(22)

In [97]:

import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model


print('Indexing word vectors.')

embeddings_index = {}
f = open('glove.6B/glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [98]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

Labels = to_categorical(np.asarray(Labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', Labels.shape)

Found 74939 unique tokens.
Shape of data tensor: (2497, 1000)
Shape of label tensor: (2497, 23)


In [99]:

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
Labels = Labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = Labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = Labels[-num_validation_samples:]

#60% train : 20% val: 20%test
#x_train = data[:-2*num_validation_samples]
#y_train = Labels[:-2*num_validation_samples]
#x_val = data[-2*num_validation_samples:-num_validation_samples]
#y_val = Labels[-2*num_validation_samples:-num_validation_samples]
#x_test = data[-num_validation_samples:]
#y_test = Labels[-num_validation_samples:]

In [100]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

Preparing embedding matrix.


In [120]:
from keras import backend as K
from keras import regularizers
from keras.layers import Dropout

def l1_reg(weight_matrix):
    return 0.01 * K.sum(K.abs(weight_matrix))

print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
#x = Dropout(0.5)(x)
x = MaxPooling1D(5)(x)
#x = Dropout(0.5)(x)
x = Conv1D(128, 5, activation='relu')(x)
#x = Dropout(0.5)(x)
x = MaxPooling1D(5)(x)
#x = Dropout(0.2)(x)
x = Conv1D(128, 5, activation='relu')(x)
#x = Dropout(0.2)(x)
x = GlobalMaxPooling1D()(x)
#x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
#preds = Dense(26, activation='softmax', kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01))(x)
preds = Dense(23, activation='softmax')(x)
model = Model(sequence_input, preds)


model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=30,
          validation_data=(x_val, y_val))

Training model.
Train on 1998 samples, validate on 499 samples
Epoch 1/30
1998/1998 [==============================] - 31s - loss: 2.9388 - acc: 0.1221 - val_loss: 2.7635 - val_acc: 0.1062
Epoch 2/30
1998/1998 [==============================] - 28s - loss: 2.7400 - acc: 0.1456 - val_loss: 2.5982 - val_acc: 0.2044
Epoch 3/30
1998/1998 [==============================] - 28s - loss: 2.6126 - acc: 0.1947 - val_loss: 2.4551 - val_acc: 0.2345
Epoch 4/30
1998/1998 [==============================] - 27s - loss: 2.4345 - acc: 0.2492 - val_loss: 2.2812 - val_acc: 0.3066
Epoch 5/30
1998/1998 [==============================] - 28s - loss: 2.2481 - acc: 0.2983 - val_loss: 2.4348 - val_acc: 0.1984
Epoch 6/30
1998/1998 [==============================] - 28s - loss: 2.1499 - acc: 0.3348 - val_loss: 2.3039 - val_acc: 0.2806
Epoch 7/30
1998/1998 [==============================] - 29s - loss: 2.0930 - acc: 0.3438 - val_loss: 1.9703 - val_acc: 0.3908
Epoch 8/30
1998/1998 [==============================] -

In [121]:
# evaluate the model
loss, accuracy = model.evaluate(x_val, y_val)
print('Accuracy: %f' % (accuracy*100))

499/499 [==============================] - 2s     
Accuracy: 47.695391


In [71]:
# evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: %f' % (accuracy*100))

2139/2139 [==============================] - 11s    
Accuracy: 49.041608
